In [1]:
# 🚀 Install necessary packages
!pip install streamlit openai h2o pandas scikit-learn pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [2]:
# Install libraries
!pip install h2o openai pandas scikit-learn

In [3]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.2
    Uninstalling openai-1.76.2:
      Successfully uninstalled openai-1.76.2


In [4]:

# 🔐 Configure your API keys
from pyngrok import ngrok, conf
import os, time, json
conf.get_default().auth_token = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"  # Replace with your ngrok token
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"    # Replace with your OpenAI API key

In [5]:
# 🛠️ Create enhanced Streamlit app with model download, Auto-Correction + Meta Monitor
with open("streamlit_app.py", "w") as f:
    f.write('''
import os
import time
import streamlit as st
import openai
import h2o
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from h2o.automl import H2OAutoML

os.environ['STREAMLIT_SERVER_HEADLESS'] = 'true'
os.environ['STREAMLIT_SERVER_PORT'] = '8501'
os.environ['STREAMLIT_SERVER_ADDRESS'] = '0.0.0.0'

h2o.init()
openai.api_key = os.getenv("OPENAI_API_KEY")

log_file = "/content/meta_monitor.json"
if not os.path.exists(log_file):
    with open(log_file, "w") as f:
        json.dump({"runs": []}, f)

st.title("🏭 Factory AI Resilience Engine")
uploaded_file = st.file_uploader("💾 Upload CSV", type="csv")

if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.success("✅ Data uploaded!")
    st.dataframe(df.head())

    target_column = st.selectbox("🎯 Select Target Column", options=df.columns, index=len(df.columns)-1)
    drop_cols = [col for col in df.columns if col.lower() in ['udi', 'product id', 'id']]
    df.drop(columns=drop_cols, inplace=True)

    from sklearn.preprocessing import LabelEncoder
    for col in df.select_dtypes(include='object').columns:
        df[col] = LabelEncoder().fit_transform(df[col])

    description = st.text_area("📋 Describe the disruption/problem:")
    run_button = st.button("🚀 Run AI Workflow")
else:
    run_button = False

if run_button:
    max_retries = 2
    accuracy_threshold = 0.95
    auc_threshold = 0.95
    retry_count = 0
    success = False

    while retry_count <= max_retries and not success:
        st.markdown(f"### 🔁 Attempt {retry_count + 1}")
        prompt = f"""
        You are a manufacturing AI strategist.
        Given the following problem, reason step-by-step:
        - Root cause
        - Recovery strategies
        - AI tasks needed (classification/regression/forecasting)
        - Model architecture suggestions

        Problem:
        {description}
        """
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "system", "content": "You are an AI strategist for manufacturing."},
                      {"role": "user", "content": prompt}],
            temperature=0.4,
            max_tokens=800
        )
        llm_output = response['choices'][0]['message']['content']
        st.code(llm_output)

        def detect_model_type(text):
            t = text.lower()
            return "classification" if "classification" in t else "regression" if "regression" in t else None

        task = detect_model_type(llm_output)
        if not task:
            st.warning("⚠️ Could not detect model type from LLM output.")
            break

        train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
        train_h2o = h2o.H2OFrame(train_df)
        test_h2o = h2o.H2OFrame(test_df)
        x = [col for col in train_h2o.columns if col != target_column]
        y = target_column
        train_h2o[y] = train_h2o[y].asfactor()
        test_h2o[y] = test_h2o[y].asfactor()

        aml = H2OAutoML(max_models=10, seed=1)
        aml.train(x=x, y=y, training_frame=train_h2o)
        perf = aml.leader.model_performance(test_data=test_h2o)

        try:
            acc = perf.accuracy()[0][1] if task == "classification" else None
            auc = perf.auc() if task == "classification" else None
        except:
            acc = auc = None

        st.success("✅ AutoML complete")
        st.text(perf)

        if acc and acc >= accuracy_threshold and auc and auc >= auc_threshold:
            success = True
            st.success(f"🎉 Model passed thresholds (Accuracy: {acc:.3f}, AUC: {auc:.3f})")
        else:
            st.warning(f"🔁 Model failed (Accuracy: {acc}, AUC: {auc}). Re-prompting LLM...")
            retry_count += 1

    # Save meta-monitor log
    with open(log_file, "r") as f:
        logs = json.load(f)
    logs["runs"].append({
        "attempt": retry_count,
        "success": success,
        "accuracy": acc,
        "auc": auc,
        "timestamp": time.time()
    })
    with open(log_file, "w") as f:
        json.dump(logs, f, indent=2)

    try:
        varimp = aml.leader.varimp(use_pandas=True)
        if varimp is not None and not varimp.empty:
            st.subheader("🔎 Feature Importance")
            st.bar_chart(varimp.head(10).set_index("variable")["relative_importance"])
        else:
            st.info("ℹ️ Feature importance not available.")
    except Exception as e:
        st.warning(f"⚠️ Could not extract feature importance: {e}")

    # 💾 Save & offer model download
    try:
        model_path = h2o.save_model(model=aml.leader, path="/content", force=True)
        st.success("💾 Best model saved.")
        with open(model_path, "rb") as f:
            st.download_button("📥 Download Trained Model", f, file_name=os.path.basename(model_path))
    except Exception as e:
        st.warning(f"⚠️ Model export failed: {e}")
''')



In [6]:
# ✅ Launch Streamlit + Ngrok
!pkill streamlit
os.environ['STREAMLIT_SERVER_HEADLESS'] = 'true'
os.environ['STREAMLIT_SERVER_PORT'] = '8501'
os.environ['STREAMLIT_SERVER_ADDRESS'] = '0.0.0.0'
get_ipython().system_raw("streamlit run streamlit_app.py &")
time.sleep(5)

public_url = ngrok.connect(8501)
print(f"🔗 Your enhanced AI Streamlit app is running at: {public_url}")

🔗 Your enhanced AI Streamlit app is running at: NgrokTunnel: "https://68fc-35-245-248-81.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# Launch the app via ngrok!pip install pyngrok streamlit openai h2o pandas scikit-learn --quietimport os, timefrom pyngrok import ngrok, confconf.get_default().auth_token = "YOUR_NGROK_AUTH_TOKEN"!pkill streamlitos.environ['STREAMLIT_SERVER_HEADLESS'] = 'true'os.environ['STREAMLIT_SERVER_PORT'] = '8501'os.environ['STREAMLIT_SERVER_ADDRESS'] = '0.0.0.0'os.system("streamlit run streamlit_app.py &")time.sleep(5)public_url = ngrok.connect(8501)print(f"🔗 App is live at: {public_url}")